**Gerekli kutuphanelerin import islemi**

In [ ]:
#Temel
import pandas as pd
import numpy as np

#Pre-processing
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob

#

**Veriyi yukleme ve bos veriden temizleme**

In [ ]:
# Load the dataset
df = pd.read_csv('src.csv')

# Handle missing values
#df = df.fillna(df.mean())
df = df.dropna()

df.to_csv('drop.csv')

**Pre-procesing adimlari: Stemming ve stopwords**

In [ ]:
# Correct typos for each word in the comment_text column
# df['comment_text'] = df['comment_text'].apply(lambda x: ' '.join([str(TextBlob(word).correct()) for word in x.split()]))
# df['comment_text'] = df['comment_text'].str.replace('teh', 'the')

# Perform stemming
stemmer = PorterStemmer()
df['comment_text'] = df['comment_text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['comment_text'] = df['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

df.to_csv('stem.csv')

**Toxicity degerlerinde kayip degerleri ortalama ile degistiriyoruz**

In [ ]:
df1 = df
# Handle missing values by replacing them with the mean value of the column
df1 = df1.fillna(df1['severe_toxicity'].mean())

# Handle typos and incorrect data types by replacing them with the correct values
df1['severe_toxicity'] = df1['severe_toxicity'].fillna(df1['severe_toxicity'].mean())

# df1.to_csv('toxicity.csv')
# print(df1['severe_toxicity'])

**Birinci görev: "is_toxic" sutunu; Toksik değeri 0’a eşit olan yorumlar icin 0, toksik değeri 0’dan 
büyük olanlar icin 1 olacak sekilde siniflanacak şekilde ayarlanacaktır.**

In [ ]:

df2 = df1
# Create a binary label for the severe_toxicity column
df2['is_toxic'] = df2['severe_toxicity'].apply(lambda x: 1 if x > 0 else 0)

df2.head(25)

**İkinci görev de ise toksik değerler için bir threshold belirleyerek, üç sınıf olacak 
şekilde bir ayar yapılması beklenmektedir. Sınıflar “toksik değil”, “hafif toksik” ve 
“aşırı toksik” olacak şekilde ayarlanacaktır. Çoklu sınıflandırma yapılacaktır.**

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

df3 = df2
# Define the model
model = LogisticRegression()

# Define the parameter grid for the grid search
param_grid = {'class_weight': [{0: 1, 1: w} for w in range(1, 10)]}

# Define the cross-validation splitter
splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the grid search object
grid_search = GridSearchCV(model, param_grid, cv=splitter, scoring='f1')

# Fit the grid search object to the data
grid_search.fit(X, y)

# Print the best threshold value
print(f'Best threshold value: {grid_search.best_params_}')


NameError: name 'StratifiedKFold' is not defined

In [42]:
# Import the necessary libraries
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
df3 = df2

# Define the model and the parameters to search over
model = LogisticRegression()
param_grid = {'class_weight': [{1: w} for w in [0.1, 0.2, 0.4, 0.5, 0.7, 0.9, 1]]}

# Use GridSearchCV to find the best threshold value
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
grid_search.fit(X, y)
best_threshold = grid_search.best_params_['class_weight']
print(f'Best threshold value: {best_threshold}')


ValueError: 
All the 35 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/enesdiler/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/enesdiler/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1138, in fit
    X, y = self._validate_data(
  File "/Users/enesdiler/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py", line 596, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/Users/enesdiler/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py", line 1074, in check_X_y
    X = check_array(
  File "/Us

In [ ]:
'''df['class'] = df['severe_toxicity'].apply(lambda x: 0 if x == 0 else 1)

X = df[['id', 'comment_text', 'severe_toxicity', 'obscene',
        'identity_attack', 'insult', 'threat', 'asian', 'atheist',
        'bisexual', 'black', 'buddhist', 'christian', 'female',
        'heterosexual', 'hindu', 'homosexual_gay_or_lesbian',
        'intellectual_or_learning_disability', 'jewish', 'latino', 'male',
        'muslim', 'other_disability', 'other_gender',
        'other_race_or_ethnicity', 'other_religion',
        'other_sexual_orientation', 'physical_disability',
        'psychiatric_or_mental_illness', 'transgender', 'white',
        'created_date', 'publication_id', 'parent_id', 'article_id',
        'rating', 'funny', 'wow', 'sad', 'likes', 'disagree',
        'sexual_explicit', 'identity_annotator_count',
        'toxicity_annotator_count']]
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)
# Make predictions on the testing data
predictions = classifier.predict(X_test)
# Evaluate the model using various performance metrics
print("Accuracy:", accuracy_score(y_test, predictions))
print("F-measure:", f1_score(y_test, predictions))
print("Precision:", precision_score(y_test, predictions))
print("Recall:", recall_score(y_test, predictions))'''


In [ ]:
'''# # Create a function that assigns a class based on the value of  "severe_toxicity"
def get_class(x):
    if x == 0:
        return "non-toxic"
    elif x > 0 and x <= 0.5:
        return "midly toxic"
    else:
        return "extremely toxic"


# # Create a new column called 'class' that indicates the class of each row
df['class'] = df['severe_toxicity'].apply(get_class)

# # Display the first few rows of the DataFrame to see the new 'class' column
df.head(30)

# display only "class" column equals to "extremely toxic" rows
df[df["class"] == "extremely toxic"]'''


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import utils

dff = df

print(dff.head(100)["severe_toxicity"])

X = df[['id']]

y = dff['severe_toxicity']

#convert y values to categorical values
lab = preprocessing.LabelEncoder()
y_transformed = lab.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression()
model.fit(X_train_scaled, y_train,1)

y_pred = model.predict(X_test_scaled)

confusion_matrix(y_test, y_pred)

f1_score(y_test, y_pred, average='macro')

print(f1_score(y_test, y_pred , average='macro'))

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9, 1.0]

best_threshold = 0
best_f1 = 0

for threshold in thresholds:
    y_pred_threshold = [1 if x >= threshold else 0 for x in y_pred]
    f1 = f1_score(y_test, y_pred_threshold, average='macro')
    if f1 > best_f1:
        best_threshold = threshold
        best_f1 = f1

print(f"Best threshold: {best_threshold}")
print(f"Best F1 score: {best_f1}")


In [ ]:

from sklearn.linear_model import LogisticRegression
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df, df["severe_toxicity"], test_size=0.2, random_state=42)

# Train a classification model on the training data
model = LogisticRegression()
model.fit(X_train, y_train)

# Use the model to predict the probabilities of severe toxicity for the test data
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Convert the predicted probabilities to binary labels using different threshold values
thresholds = np.arange(0, 1.1, 0.1)
y_preds = [np.array([1 if p >= t else 0 for p in y_pred_proba])
           for t in thresholds]

# Calculate the F1 score for each threshold value
scores = [f1_score(y_test, y_pred) for y_pred in y_preds]

# Find the threshold that results in the highest F1 score
best_threshold = thresholds[np.argmax(scores)]
print(f"Best threshold: {best_threshold}")


In [ ]:
# Set the true labels and predicted probabilities for your data
y_true = [0, 1, 0, 1, 0, 1]
y_pred_proba = [0.1, 0.9, 0.2, 0.8, 0.3, 0.7]

# Convert the predicted probabilities to binary labels using different threshold values
thresholds = np.arange(0, 1.1, 0.1)
y_preds = [np.array([1 if p >= t else 0 for p in y_pred_proba])
           for t in thresholds]

# Calculate the F1 score for each threshold value
scores = [f1_score(y_true, y_pred) for y_pred in y_preds]

# Find the threshold that results in the highest F1 score
best_threshold = thresholds[np.argmax(scores)]
print(f"Best threshold: {best_threshold}")


In [ ]:
from sklearn.linear_model import LogisticRegression

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df["severe_toxicity"], df["target"], test_size=0.2)

# Train a model on the training data
model =  LogisticRegression() # or some other model
model.fit(X_train, y_train)

# Predict probabilities on the test data
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Convert the predicted probabilities to binary labels using different threshold values
thresholds = np.arange(0, 1.1, 0.1)
y_preds = [np.array([1 if p >= t else 0 for p in y_pred_proba])
           for t in thresholds]

# Calculate the F1 score for each threshold value
scores = [f1_score(y_test, y_pred) for y_pred in y_preds]

# Find the threshold that results in the highest F1 score
best_threshold = thresholds[np.argmax(scores)]
print(f"Best threshold: {best_threshold}")


In [ ]:
# Set the true labels and predicted probabilities for the "severe_toxicity" column
y_true = df["severe_toxicity"]
y_pred_proba = df["severe_toxicity_predicted_probability"]

# Convert the predicted probabilities to binary labels using different threshold values
thresholds = np.arange(0, 1.1, 0.1)
y_preds = [np.array([1 if p >= t else 0 for p in y_pred_proba])
           for t in thresholds]

# Calculate the F1 score for each threshold value
scores = [f1_score(y_true, y_pred) for y_pred in y_preds]

# Find the threshold that results in the highest F1 score
best_threshold = thresholds[np.argmax(scores)]
print(f"Best threshold: {best_threshold}")


In [ ]:
# Define the classification function
def classify_toxicity(x):
    if x < 0.5:
        return 'non-toxic'
    elif x < 0.75:
        return 'mildly toxic'
    else:
        return 'extremely toxic'

# Apply the classification function to the "severe_toxicity" column
df['class'] = df['severe_toxicity'].apply(classify_toxicity)

# Preview the data
df.head()

# Split the data into training and testing sets
X = df.drop(['class', 'severe_toxicity'], axis=1)
y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model's performance
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score

X = df.drop(columns=['id', 'severe_toxicity'])
y = df['severe_toxicity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a random forest classifier on the training set
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the test set and compute the precision-recall curve
y_pred = clf.predict_proba(X_test)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)

# Plot the precision-recall curve
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

# Compute the area under the curve (AUC)
auc_score = auc(recall, precision)
print(f'AUC: {auc_score:.2f}')

# Set the threshold value to be the point on the curve with the highest precision
threshold = thresholds[np.argmax(precision)]
print(f'Threshold: {threshold:.2f}')
